## 变形

### 长宽表的变形

#### 长宽表

- 长表和宽表是以一个特征为基准进行定义的, 如果一个特征被作为内容计入了表内, 那么这个表是长表, 如果其作为表的索引来对其他特征的数据进行分类, 那么这种表就是这种特征的宽表.

而长表和宽表很多时候都是等价的, 它们储存了相同的信息, 只是表达方式不同, 因此可以通过一些函数进行长宽表的转换.

#### pivot

pivot 是典型的长表变宽表的函数. 利用 pivot 可以将一个列表中某一列作为列, 另一列作为索引, 然后得到关于这些特征的宽表.

pivot 的参数中最重要的是变形后的行索引、需要转到列索引的列，以及这些列和行索引对应的数值:  index, columns, values
![](https://s1.vika.cn/space/2022/09/10/0d46a076c805487fa0a5d2a0ae0af05d)

pivot 的变形要求数据具有唯一性: 原表中的 `index` 和 `columns` 对应两个列的行组合必须唯一.

#### pivot_table

pivot_table 相对于 pivot 函数而言, 其可以忽视唯一性的要求, 其可以通过聚合操作使得相同行列组合对应的多个值变为一个值, 聚合方法可以在参数 aggfunc 中进行设置.'

pivot_table 同样可以设置 margins 参数来打开边际汇总功能, 即会统计一个特征整体的统计数据, 具体的数据内容由 aggfunc 决定.

#### melt

作为 pivot 的反函数, melt 是将宽表转化为宽表, 参数与 pivot 相同, 使用 index, columns, values 三种参数来拆解宽表.

![](https://s1.vika.cn/space/2022/09/10/2b859851570b44d19b10a2b990498c2b)

#### wide_to_long

如果列中包含了交叉类别，比如期中期末的类别和语文数学的类别，那么想要把 `values_name` 对应的 `Grade` 扩充为两列分别对应语文分数和数学分数，只把期中期末的信息压缩，这种需求下就要使用 `wide_to_long` 函数来完成.

### 索引的变形

#### stack & unstack

使用 stack 和 unstack 可以在行列索引间进行索引转换.

unstack 将行索引转为列索引, stack 将列索引转为行索引.

两个函数的默认值是1, 从对应索引的最里面一层进行转化, 可以通过自定义层数来调整变形的对象.

### 其他变形函数

#### explode

`explode` 参数能够对某一列的元素进行纵向的展开, 被展开的单元格必须存储 `list, tuple, Series, np.ndarray` 中的一种类型

#### get_dummies

`get_dummies` 是用于特征构建的重要函数之一, 其作用是把类别特征转为指示变量.



In [1]:
import pandas as pd
import numpy as np

## 四、练习
### Ex1：美国非法药物数据集

现有一份关于美国非法药物的数据集，其中`SubstanceName, DrugReports`分别指药物名称和报告数量：

In [5]:
df = pd.read_csv('./data/drugs.csv').sort_values(['State','COUNTY','SubstanceName'],ignore_index=True)
df.head(3)

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4


1. 将数据转为如下的形式：

    ![](https://s1.vika.cn/space/2022/09/10/348eeee0472a46c187fdcced4c3a70a1)

2. 将第1问中的结果恢复为原表
3. 按 State 分别统计每年的报告数量总和，其中 State, YYYY 分别为列索引和行索引，要求分别使用 pivot_table 函数与 groupby+unstack 两种不同的策略实现，并体会它们之间的联系

In [13]:
res1 = df.pivot(index=['State', 'COUNTY', 'SubstanceName'], columns='YYYY', values='DrugReports')
res1 = res1.reset_index().rename_axis(columns={'YYYY': ''})
res1

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,NaN,3.0,5.0,4.0,27.0,5.0,7.0,10.0
1,KY,ADAIR,Codeine,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,KY,ADAIR,Fentanyl,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,KY,ADAIR,Heroin,NaN,NaN,1.0,2.0,NaN,1.0,NaN,2.0
4,KY,ADAIR,Hydrocodone,6.0,9.0,10.0,10.0,9.0,7.0,11.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
6209,WV,WOOD,Oxycodone,6.0,4.0,24.0,7.0,7.0,11.0,7.0,1.0
6210,WV,WOOD,Tramadol,NaN,NaN,NaN,NaN,1.0,NaN,4.0,3.0
6211,WV,WYOMING,Buprenorphine,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0
6212,WV,WYOMING,Hydrocodone,1.0,5.0,NaN,NaN,1.0,NaN,1.0,NaN


In [29]:
res2 = res1.melt(id_vars=['State', 'COUNTY', 'SubstanceName'], value_vars=res1[-8:], var_name='YYYY', value_name='DrugReports')
res2 = res2.dropna(subset='DrugReports')
res2 = res2[df.columns].sort_values(['State','COUNTY','SubstanceName'],ignore_index=True).astype({'YYYY':'int64', 'DrugReports':'int64'})
res2

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4
3,2014,KY,ADAIR,Buprenorphine,27
4,2015,KY,ADAIR,Buprenorphine,5
...,...,...,...,...,...
24057,2010,WV,WYOMING,Oxycodone,5
24058,2011,WV,WYOMING,Oxycodone,4
24059,2012,WV,WYOMING,Oxycodone,14
24060,2013,WV,WYOMING,Oxycodone,12


In [23]:
res3 = df.pivot_table(values='DrugReports', index='YYYY', columns='State', aggfunc='sum')
res3

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376
2013,11148,26846,20409,11675,4046
2014,11081,30860,24904,9037,3280
2015,9865,37127,25651,8810,2571
2016,9093,42470,26164,10195,2548
2017,9394,46104,27894,10448,1614


In [39]:
res4 = df.groupby(['YYYY', 'State'])['DrugReports'].sum().to_frame().unstack(1).droplevel(0, axis=1)
res4

State,KY,OH,PA,VA,WV
YYYY,,,,,
2010,10453,19707,19814,8685,2890
2011,10289,20330,19987,6749,3271
2012,10722,23145,19959,7831,3376
2013,11148,26846,20409,11675,4046
2014,11081,30860,24904,9037,3280
2015,9865,37127,25651,8810,2571
2016,9093,42470,26164,10195,2548
2017,9394,46104,27894,10448,1614


### Ex2：特殊的wide_to_long方法

从功能上看，`melt`方法应当属于`wide_to_long`的一种特殊情况，即`stubnames`只有一类。请使用`wide_to_long`生成`melt`一节中的`df_melted`。（提示：对列名增加适当的前缀）

In [3]:
df = pd.DataFrame({'Class':[1,2],
                   'Name':['San Zhang', 'Si Li'],
                   'Chinese':[80, 90],
                   'Math':[80, 75]})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75
